In [1286]:
import numpy as np                        
from qiskit import *                 
from qiskit_aer import *             
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import *

In [1287]:
#user_num = int(input("Enter the max for your QRNG range:"))   # User input for max range 
#num_qubits = int(np.ceil(np.log2(user_num)))                  # Obtains number of qubits for the circuit
num_qubits = 31
division_size = 10

In [1288]:
# Generates random number dict
def number_generator_simulator(num_qubits):
    circ = QuantumCircuit(num_qubits, num_qubits)                 # Creates circuit with number of qubits obtained
    circ.h(range(num_qubits))                                     # Applies a hadamard gate to all qubits
    circ.measure(range(num_qubits), range(num_qubits))            # Measures all qubits and assigns them to classical bits

    simulator = AerSimulator()                                         # Lets us use the Aer Simulator 
    compiled_circuit = transpile(circ, simulator)                      # Compiled circuit using Aer 
    result = simulator.run(compiled_circuit, shots = 50).result()      # Result with the 10000 shots as to not run forever on IBM machines
    counts = result.get_counts()                                       # Assigns counts with the result
    
    return counts

In [1289]:
# For mod 2 method
def obtain_rand(counts):
    # Get all values
    max_value = max(counts.values())
    max_keys = [key for key, value in counts.items() if value == max_value]
    max_dict = {key: counts[key] for key in max_keys}
    # XOR all ties together to get new random
    new_key = [int(i) for i in max_keys[0]]
    for i in range(1, len(max_keys) - 1):
        for j in range (0, num_qubits):
            new_key[j] += int(max_keys[i][j])

    super_new_key = ''
    # Bitwise XOR (mod % 2) on each qubit
    for i in range (0, num_qubits):
        # super_new_key[i] = str(int(new_key[i]) % 2)
        super_new_key += str(int(new_key[i]) % 2)

    return super_new_key

In [1290]:
# For iterative method
def obtain_ties(counts):
    max_value = max(counts.values())
    max_keys = [key for key, value in counts.items() if value == max_value]
    max_dict = {key: counts[key] for key in max_keys}

    while (len(max_keys) > 1):
        # print(max_keys)
        max_keys = tie_breaker(max_keys)

    return max_keys
        

def tie_breaker(ties):
    new_num_qubits = int(np.ceil(np.log2(len(ties))))
    counts = number_generator_simulator(new_num_qubits)
    max_value = max(counts.values()) # Max among the ties
    max_keys = [key for key, value in counts.items() if value == max_value] # Keys (indexes!) that tied
    max_dict = {key: counts[key] for key in max_keys} # Dictionary of tied index->value's
    # print("Max key: ", max_keys)
    
    # Need to use new ties (indexes of original ties) to index which ties make it
    new_ties = []
    for i in range (0, len(max_keys)):
        new_ties.append(ties[int(max_keys[i],2)])
    # new_ties = [indexKey for indexKey, value in max_dict if ]

    return new_ties

In [1291]:
# For concatenation method
def concatenation(num_qubits, division_size):
    extra = num_qubits % division_size
    iterations = int((num_qubits - extra) / division_size)
    final_output = []

    for iteration in range(iterations):
        division_chunk = None

        while division_chunk is None:
            counts = number_generator_simulator(division_size)
            division_chunk = check_tie(counts)
        final_output.append(division_chunk)

    if extra != 0:
        division_chunk = None

        while division_chunk is None:
            counts = number_generator_simulator(extra)
            division_chunk = check_tie(counts)
        final_output.append(division_chunk)

    return ''.join(final_output)

def check_tie(counts):
    max_value = max(counts.values())
    max_keys = [key for key, value in counts.items() if value == max_value]

    if len(max_keys) > 1:
        return None
    else:
        return max_keys[0]

In [1292]:
# Using concatenation method
final_val = concatenation(num_qubits, division_size)
print(f'Final random qubit state:', final_val)

Final random qubit state: 1111101010100010111100010100110


In [1293]:
# Using iterative method
for i in range (0, 10):
    counts = number_generator_simulator(num_qubits)
    final_val = obtain_ties(counts)
    print(f'Final random qubit state:', final_val)

CircuitTooWideForTarget: 'Number of qubits (31) in circuit-2049 is greater than maximum (30) in the coupling_map'

In [167]:
# Using mod 2 method
for i in range (0, 100):
    counts = number_generator_simulator(num_qubits)
    rand_value = obtain_rand(counts)
    # print(f'Data:', counts)
    print(f'Final random qubit state:', rand_value)

Final random qubit state: 110101111111010011011101111010
Final random qubit state: 100101111010001110011101010001
Final random qubit state: 010101100101000000000110010110
Final random qubit state: 011000100100010010111011111001
Final random qubit state: 101001010000000110000110001110
Final random qubit state: 101110010001010000010001011000
Final random qubit state: 010000100111011001011111110101
Final random qubit state: 111001101001010100010110100100
Final random qubit state: 010111001100000110101110111101
Final random qubit state: 110000011010101001011100111100
Final random qubit state: 110110110000110100110111101100
Final random qubit state: 011101110011101010011011101101
Final random qubit state: 011111111111011000001011101011
Final random qubit state: 011010001001101110100010111000
Final random qubit state: 011001100001011110010100111011
Final random qubit state: 000001100100001100010010100100
Final random qubit state: 100011011001101000001110000001
Final random qubit state: 00101